# Basics

This notebook provides a quick introduction on how to get started with loading a simulation output and creating some basic figures.

The data files used in these documentation pages can be found 

## Reading a RAMSES output

Import the `osyris` module and load the output of your choice.
We are loading output number `8` from a simulation of a collapsing system of gas,
that is forming young stars in its core.

<div class="alert alert-warning">

**IMPORTANT**

The data loader searches for descriptor files (`hydro_file_descriptor.txt`, `rt_file_descriptor` and the like)
inside the output directory to get the variable names.

If your version of RAMSES does not support this (the file descriptor did not exist in old versions of RAMSES),
you will need to generate one and copy it into each of your output directories.

</div>

In [ ]:
import osyris
import numpy as np

In [ ]:
data = osyris.Dataset(8, path="osyrisdata/starformation")

When creating a `Dataset`, the first argument is the output number. 
**Note** that you can use `-1` to select the last output in the directory. 

Upon creation, the `Dataset` only reads the simulation metadata,
and does not load the bulk of the simulation data.

To actually load the data, we call the `load()` method on the `Dataset`.
The `Dataset` has a `__str__` representation,
which will return a list of all the variables contained in the loaded file,
along with their minimum and maximum values.

In [ ]:
data.load()

The `Dataset` behaves very much like a Python dictionary.
The data it contains is divided into `Datagroups`,
which can be accessed just like accessing elements of a dictionary.

In [ ]:
data.keys()

To access a single variable array, we access entries in a `Datagroup` in the same way:

In [ ]:
data["hydro"]["density"]

The information printed is the name of the variable, its minimum and maximum value,
its physical unit, and the number of elements (or cells) in the array.

## Creating a 2D histogram

We now wish to plot a 2d histogram of the logarithm of density versus logarithm of magnetic field magnitude,
for all the cells inside the computational domain.
We also use a logarithmic colormap which represents the cell counts in the $(\rho,B)$ plane

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True)

## Simple 2D map

The goal here is to create a 2D gas density slice, 2000 au wide, through the plane normal to `z`,
that passes through the center of the young star forming system.

We first need to define where the center of the system is.
A simple definition is to use the coordinate of the cell with the highest density in the simulation.

In [ ]:
ind = np.argmax(data["hydro"]["density"])
center = data["amr"]["position"][ind]
center

Once we have this center coordinate, we can use it as the origin for our cut plane and create a map:

In [ ]:
osyris.map({"data": data["hydro"]["density"], "norm": "log"},
           dx=2000 * osyris.units('au'),
           origin=center,
           direction="z")

where the first argument is the variable to display, `dx` is the extent of the viewing region,
and `direction` is the normal to the slicing plane.